In [1]:
!pip install vecstack


  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19861 sha256=7428a64fd148bb428a31cb0af535b9fc49948d676398e58c706c638c86f01c91
  Stored in directory: /root/.cache/pip/wheels/b8/d8/51/3cf39adf22c522b0a91dc2208db4e9de4d2d9d171683596220
Successfully built vecstack


In [2]:
# import the libraries
import pandas as pd
import numpy as np

from vecstack import stacking

from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter

from sklearn.preprocessing import OneHotEncoder

In [3]:
# read the training data
train_data = pd.read_csv('Cell Phone Churn-TRAIN.csv')
train_data

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,...,INCOME,SPORTS,NATURE,ARTS,HRS_TV,TRAVEL,EDUC,TOTMOU,TOTCHNG,TARGET
0,105,15,99,90,-84,6,11,0,0,2,...,51000,B,1,N,10,None,Bachelors,219,6,0
1,999,1200,1145,-201,55,-146,3,1,1,2,...,80000,A,0,N,40,Domestic,Masters,3344,-146,1
2,5,2,1,3,1,4,23,0,0,1,...,80000,C,0,Y,50,International,HighSchool,8,4,1
3,600,456,398,144,58,202,13,0,1,1,...,88000,A,1,N,0,International,Masters,1454,202,1
4,2,0,0,0,0,0,1,0,0,1,...,29000,B,0,N,75,Domestic,HighSchool,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,333,277,144,56,133,189,9,0,1,1,...,44000,F,1,N,35,International,Masters,754,189,1
1691,550,100,500,450,-400,50,17,1,1,1,...,320000,D,0,Y,50,InterGalactic,GED,1150,50,0
1692,398,144,630,254,-486,-232,36,0,0,2,...,145000,E,1,N,40,Domestic,Masters,1172,-232,0
1693,5,15,99,-10,-84,-94,11,0,0,2,...,51000,B,1,N,10,None,Bachelors,119,-94,0


In [4]:
# read the testing data
test_data = pd.read_csv('Cell Phone Churn-TEST.csv')
test_data

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,...,INCOME,SPORTS,NATURE,ARTS,HRS_TV,TRAVEL,EDUC,TOTMOU,TOTCHNG,TARGET
0,344,232,455,112,-223,-111,11,1,1,2,...,75000,B,1,Y,25,InterGalactic,GED,1031,-111,0
1,555,444,678,111,-234,-123,27,1,1,2,...,134000,na,1,Y,5,Domestic,PhD,1677,-123,0
2,2,0,4,1,-3,-2,4,0,0,1,...,53000,na,1,Y,125,None,Bachelors,6,-2,0
3,5678,3457,2156,2221,1301,3522,14,1,1,1,...,33000,A,1,Y,25,Domestic,Masters,11291,3522,0
4,333,122,144,211,-22,189,9,0,1,1,...,44000,F,1,N,35,International,Masters,599,189,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2,0,0,0,0,0,1,0,0,1,...,29000,B,0,N,75,Domestic,HighSchool,0,0,0
371,53,34,29,19,5,24,50,1,1,1,...,48000,na,1,Y,20,None,GED,116,24,1
372,99,2,0,97,0,0,2,0,0,1,...,17000,F,1,Y,70,Domestic,Bachelors,0,0,0
373,78,54,24,24,30,54,6,1,0,1,...,66000,A,0,N,20,International,Bachelors,156,54,1


In [5]:
# note: test data also contains the target column. no need to use train_test_split.

In [6]:
# data exploration
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MOUMO    1695 non-null   int64 
 1   MOUPMO   1695 non-null   int64 
 2   MOU3MO   1695 non-null   int64 
 3   MOUCH1M  1695 non-null   int64 
 4   MOUCH3M  1695 non-null   int64 
 5   CUM3MCH  1695 non-null   int64 
 6   CUSTMOS  1695 non-null   int64 
 7   LONGD    1695 non-null   int64 
 8   CALLW    1695 non-null   int64 
 9   LINES    1695 non-null   int64 
 10  VOICEM   1695 non-null   int64 
 11  CELL     1695 non-null   int64 
 12  CONVB    1695 non-null   object
 13  SEX      1695 non-null   object
 14  INCOME   1695 non-null   int64 
 15  SPORTS   1695 non-null   object
 16  NATURE   1695 non-null   int64 
 17  ARTS     1695 non-null   object
 18  HRS_TV   1695 non-null   int64 
 19  TRAVEL   1695 non-null   object
 20  EDUC     1695 non-null   object
 21  TOTMOU   1695 non-null   int64 
 22  

In [7]:
# takeaway: no missing values, mix of categorical and numeric columns

In [8]:
train_data['TARGET'].value_counts()

0    1052
1     643
Name: TARGET, dtype: int64

In [9]:
# takeaway: dataset is imbalanced.

In [10]:
X_train = train_data.drop('TARGET', axis=1)
y_train = train_data['TARGET']
X_test = test_data.drop('TARGET', axis=1)
y_test = test_data['TARGET']

In [11]:
# perform one hot encoding on the training and testing dataset to convert categorical columns to numeric columns.

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([X_train,X_test], keys=[0,1])

#Do one Hot encoding for categorical features
categorical_columns = []
for col in combined_Data.columns:
  if combined_Data[col].dtype=='O':
    categorical_columns.append(col)


ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
combined_cat = pd.DataFrame(ohe.fit_transform(combined_Data[categorical_columns]),
                             columns=ohe.get_feature_names_out(),
                             index=combined_Data.index)
combined_Data = pd.concat([combined_Data,combined_cat],axis=1)
combined_Data.drop(labels = categorical_columns,axis=1,inplace=True)
combined_Data

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MOUMO  MOUPMO  MOU3MO  MOUCH1M  MOUCH3M  CUM3MCH  CUSTMOS  LONGD  \
0 0      105      15      99       90      -84        6       11      0   
  1      999    1200    1145     -201       55     -146        3      1   
  2        5       2       1        3        1        4       23      0   
  3      600     456     398      144       58      202       13      0   
  4        2       0       0        0        0        0        1      0   
...      ...     ...     ...      ...      ...      ...      ...    ...   
1 370      2       0       0        0        0        0        1      0   
  371     53      34      29       19        5       24       50      1   
  372     99       2       0       97        0        0        2      0   
  373     78      54      24       24       30       54        6      1   
  374    999    1200    1145     -201       55     -146        3      1   

       CALLW  LINES  ...  TRAVEL_Domestic  TRAVEL_InterGalactic  \
0 0        0      2  ...              0.0                   0.0   
  1        1      2  ...              1.0                   0.0   
  2        0      1  ...              0.0                   0.0   
  3        1      1  ...              0.0                   0.0   
  4        0      1  ...              1.0                   0.0   
...      ...    ...  ...              ...                   ...   
1 370      0      1  ...              1.0                   0.0   
  371      1      1  ...              0.0                   0.0   
  372      0      1  ...              1.0                   0.0   
  373      0      1  ...              0.0                   0.0   
  374      1      2  ...              1.0                   0.0   

       TRAVEL_International  TRAVEL_None  EDUC_Bachelors  EDUC_Elementary  \
0 0                     0.0          1.0             1.0              0.0   
  1                     0.0          0.0             0.0              0.0   
  2                     1.0          0.0             0.0              0.0   
  3                     1.0          0.0             0.0              0.0   
  4                     0.0          0.0             0.0              0.0   
...                     ...          ...             ...              ...   
1 370                   0.0          0.0             0.0              0.0   
  371                   0.0          1.0             0.0              0.0   
  372                   0.0          0.0             1.0              0.0   
  373                   1.0          0.0             1.0              0.0   
  374                   0.0          0.0             0.0              0.0   

       EDUC_GED  EDUC_HighSchool  EDUC_Masters  EDUC_PhD  
0 0         0.0              0.0           0.0       0.0  
  1         0.0              0.0           1.0       0.0  
  2         0.0              1.0           0.0       0.0  
  3         0.0              0.0           1.0       0.0  
  4         0.0              1.0           0.0       0.0  
...         ...              ...           ...       ...  
1 370       0.0              1.0           0.0       0.0  
  371       1.0              0.0           0.0       0.0  
  372       0.0              0.0           0.0       0.0  
  373       0.0              0.0           0.0       0.0  
  374       0.0              0.0           0.0       1.0  

[2070 rows x 40 columns]

In [12]:
# getting the train and test test back
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)


In [13]:
X_train

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,...,TRAVEL_Domestic,TRAVEL_InterGalactic,TRAVEL_International,TRAVEL_None,EDUC_Bachelors,EDUC_Elementary,EDUC_GED,EDUC_HighSchool,EDUC_Masters,EDUC_PhD
0,105,15,99,90,-84,6,11,0,0,2,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,999,1200,1145,-201,55,-146,3,1,1,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5,2,1,3,1,4,23,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,600,456,398,144,58,202,13,0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2,0,0,0,0,0,1,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,333,277,144,56,133,189,9,0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1691,550,100,500,450,-400,50,17,1,1,1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1692,398,144,630,254,-486,-232,36,0,0,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1693,5,15,99,-10,-84,-94,11,0,0,2,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_train

0       0
1       1
2       1
3       1
4       0
       ..
1690    1
1691    0
1692    0
1693    0
1694    1
Name: TARGET, Length: 1695, dtype: int64

In [15]:
X_test

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,...,TRAVEL_Domestic,TRAVEL_InterGalactic,TRAVEL_International,TRAVEL_None,EDUC_Bachelors,EDUC_Elementary,EDUC_GED,EDUC_HighSchool,EDUC_Masters,EDUC_PhD
0,344,232,455,112,-223,-111,11,1,1,2,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,555,444,678,111,-234,-123,27,1,1,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,0,4,1,-3,-2,4,0,0,1,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5678,3457,2156,2221,1301,3522,14,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,333,122,144,211,-22,189,9,0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2,0,0,0,0,0,1,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
371,53,34,29,19,5,24,50,1,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
372,99,2,0,97,0,0,2,0,0,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
373,78,54,24,24,30,54,6,1,0,1,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [16]:
y_test

0      0
1      0
2      0
3      0
4      1
      ..
370    0
371    1
372    0
373    1
374    1
Name: TARGET, Length: 375, dtype: int64

In [17]:
# sampling the data to balance the dataset using SMOTE

sm = SMOTE(random_state=19)

X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

In [18]:
X_resampled

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,...,TRAVEL_Domestic,TRAVEL_InterGalactic,TRAVEL_International,TRAVEL_None,EDUC_Bachelors,EDUC_Elementary,EDUC_GED,EDUC_HighSchool,EDUC_Masters,EDUC_PhD
0,105,15,99,90,-84,6,11,0,0,2,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,999,1200,1145,-201,55,-146,3,1,1,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5,2,1,3,1,4,23,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,600,456,398,144,58,202,13,0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2,0,0,0,0,0,1,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,78,54,24,24,30,54,6,1,0,1,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2100,678,1222,598,-544,624,80,30,1,1,2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2101,333,277,144,56,133,189,9,0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2102,53,34,29,19,5,24,50,1,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
X_test

,MOUMO,MOUPMO,MOU3MO,MOUCH1M,MOUCH3M,CUM3MCH,CUSTMOS,LONGD,CALLW,LINES,...,TRAVEL_Domestic,TRAVEL_InterGalactic,TRAVEL_International,TRAVEL_None,EDUC_Bachelors,EDUC_Elementary,EDUC_GED,EDUC_HighSchool,EDUC_Masters,EDUC_PhD
0,344,232,455,112,-223,-111,11,1,1,2,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,555,444,678,111,-234,-123,27,1,1,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,0,4,1,-3,-2,4,0,0,1,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5678,3457,2156,2221,1301,3522,14,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,333,122,144,211,-22,189,9,0,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2,0,0,0,0,0,1,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
371,53,34,29,19,5,24,50,1,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
372,99,2,0,97,0,0,2,0,0,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
373,78,54,24,24,30,54,6,1,0,1,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [20]:
y_test

0      0
1      0
2      0
3      0
4      1
      ..
370    0
371    1
372    0
373    1
374    1
Name: TARGET, Length: 375, dtype: int64

In [21]:
y_resampled.value_counts()

0    1052
1    1052
Name: TARGET, dtype: int64

In [24]:
# MLP
mlp = MLPClassifier()
mlp.fit(X_resampled, y_resampled)
mlp_predict=mlp.predict(X_test)
print("accuracy Score (training) for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test,y_test)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test,mlp_predict))
print("Classification report: ")
print(classification_report(mlp_predict, y_test))

accuracy Score (training) for MultiLayer Perceptron:0.544000
Confusion Matrix for MultiLayer Perceptron:
[[ 67 168]
 [  3 137]]
Classification report: 
              precision    recall  f1-score   support

           0       0.29      0.96      0.44        70
           1       0.98      0.45      0.62       305

    accuracy                           0.54       375
   macro avg       0.63      0.70      0.53       375
weighted avg       0.85      0.54      0.58       375



In [25]:
# SVC
svm = SVC()
svm.fit(X_resampled, y_resampled)
svm_predict=svm.predict(X_test)
print("accuracy Score (training) for SVM Classifier:{0:6f}".format(svm.score(X_test,y_test)))
print("Confusion Matrix for SVM Classifier:")
print(confusion_matrix(y_test,svm_predict))
print("Classification report: ")
print(classification_report(svm_predict, y_test))

accuracy Score (training) for SVM Classifier:0.648000
Confusion Matrix for SVM Classifier:
[[109 126]
 [  6 134]]
Classification report: 
              precision    recall  f1-score   support

           0       0.46      0.95      0.62       115
           1       0.96      0.52      0.67       260

    accuracy                           0.65       375
   macro avg       0.71      0.73      0.65       375
weighted avg       0.81      0.65      0.66       375



In [23]:
# DT
dt = DecisionTreeClassifier()
dt.fit(X_resampled, y_resampled)
train_pred = dt.predict(X_resampled)
test_pred = dt.predict(X_test)

#Model Accuracy
print("Train Accuracy:", accuracy_score(y_resampled, train_pred))
print("Test Accuracy:", accuracy_score(y_test, test_pred))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(y_test, test_pred))
print("Max Depth",dt.get_depth())
print("Leaf",dt.get_n_leaves())
print('Printing the precision and recall, among other metrics')
print(classification_report(y_test, test_pred))

Train Accuracy: 0.9881178707224335
Test Accuracy: 0.9946666666666667
Confusion Matrix for Decision Tree:
[[235   0]
 [  2 138]]
Max Depth 7
Leaf 19
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       235
           1       1.00      0.99      0.99       140

    accuracy                           0.99       375
   macro avg       1.00      0.99      0.99       375
weighted avg       0.99      0.99      0.99       375



In [22]:
# KNN
neigh = KNeighborsClassifier()



neigh.fit(X_resampled, y_resampled)
neigh_predict=neigh.predict(X_test)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,neigh_predict))
print("classification report:")
print(classification_report(neigh_predict, y_test))

accuracy Score (training) for KNeighborsClassifier:0.994667
Confusion Matrix for KNeighborsClassifier:
[[235   0]
 [  2 138]]
classification report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       237
           1       0.99      1.00      0.99       138

    accuracy                           0.99       375
   macro avg       0.99      1.00      0.99       375
weighted avg       0.99      0.99      0.99       375



In [26]:
# DT hyperparameter tuning using random search
#Hyperparameter tuning done for decision tree classifier

#RANDOM SEARCH

import time
start_time = time.time()

print("RandomizedSearchCV-Decision tree")

parameters={
            'max_depth': range(3,11),
            'criterion':['gini','entropy'],
            'max_leaf_nodes':range(5,21,3),
           }

dt_random = RandomizedSearchCV(dt,parameters,n_iter=25,cv=5)

dt_random.fit(X_resampled, y_resampled)

grid_parm = dt_random.best_params_
print(grid_parm)

train_pred_2 = dt_random.predict(X_resampled)
test_pred_2 = dt_random.predict(X_test)

print("Train Accuracy:", dt_random.score(X_resampled, y_resampled))
print("Test Accuracy:", dt_random.score(X_test, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Decision tree
{'max_leaf_nodes': 20, 'max_depth': 8, 'criterion': 'gini'}
Train Accuracy: 0.9881178707224335
Test Accuracy: 0.9946666666666667
--- 0.8212080001831055 seconds ---


In [27]:
dtRand = DecisionTreeClassifier(**grid_parm)
dtRand.fit(X_resampled,y_resampled)

dtRand_predict_train = dtRand.predict(X_resampled)
dtRand_predict = dtRand.predict(X_test)
print("Train Accuracy:", accuracy_score(y_resampled,dtRand_predict_train))
print("Test Accuracy:", accuracy_score(y_test,dtRand_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(y_test,dtRand_predict))
print('Printing the precision and recall, among other metrics')
print(classification_report(y_test, dtRand_predict))
clf_cv_score = cross_val_score(dtRand, X_resampled, y_resampled, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Train Accuracy: 0.9881178707224335
Test Accuracy: 0.9946666666666667
Confusion Matrix for Decision Tree:
[[235   0]
 [  2 138]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       235
           1       1.00      0.99      0.99       140

    accuracy                           0.99       375
   macro avg       1.00      0.99      0.99       375
weighted avg       0.99      0.99      0.99       375

[0.98574814 0.98572557 0.98574814 0.99287971 0.98809524]


In [28]:
# getting the top 5 features
print(dtRand.feature_importances_)

#print sorted list of important features
importances = dtRand.feature_importances_
indices = np.argsort(importances)[-5:] #the list is in ascending order
print("\n top 5 features = ", indices, "\n")

[1.06175518e-02 5.52015802e-02 0.00000000e+00 0.00000000e+00
 5.64465258e-01 0.00000000e+00 2.64895013e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.02642566e-01 0.00000000e+00 2.46022135e-02 3.03635679e-03
 9.11388009e-02 3.90061674e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.14161107e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]

 top 5 features =  [ 6  1 16 12  4] 



In [30]:
X_resampled.iloc[:,indices].columns

Index(['CUSTMOS', 'MOUPMO', 'TOTCHNG', 'INCOME', 'MOUCH3M'], dtype='object')

In [31]:
# performing stacking

models = [MLPClassifier(), SVC(), DecisionTreeClassifier(), KNeighborsClassifier()]

S_Train, S_Test = stacking(models,
                           X_resampled, y_resampled, X_test,
                           regression=False,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           metric=accuracy_score,

                           n_folds=4,

                           stratified=True,

                           shuffle=True,

                           random_state=0,

                           verbose=2)


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [MLPClassifier]
    fold  0:  [0.67680608]
    fold  1:  [0.54372624]
    fold  2:  [0.67870722]
    fold  3:  [0.61406844]
    ----
    MEAN:     [0.62832700] + [0.05533756]
    FULL:     [0.62832700]

model  1:     [SVC]
    fold  0:  [0.69391635]
    fold  1:  [0.74524715]
    fold  2:  [0.73764259]
    fold  3:  [0.69581749]
    ----
    MEAN:     [0.71815589] + [0.02345329]
    FULL:     [0.71815589]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.98288973]
    fold  1:  [0.99049430]
    fold  2:  [0.99049430]
    fold  3:  [0.98859316]
    ----
    MEAN:     [0.98811787] + [0.00311665]
    FULL:     [0.98811787]

model  3:     [KNeighborsClassifier]
    fold  0:  [0.97338403]
    fold  1:  [0.96197719]
    fold  2:  [0.99049430]
    fold  3:  [0.98859316]
    ----
    MEAN:     [0.97861217] + [0.01167113]
    FULL:     [0.97861217]

In [32]:
model = RandomForestClassifier()

model = model.fit(S_Train, y_resampled)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
print("Confusion Matrix for Random Forest Stacked model:")
print(confusion_matrix(y_test,y_pred))
print("classification report:")
print(classification_report(y_pred, y_test))

Final prediction score for ensemble methods: [0.99466667]
Confusion Matrix for Random Forest Stacked model:
[[235   0]
 [  2 138]]
classification report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       237
           1       0.99      1.00      0.99       138

    accuracy                           0.99       375
   macro avg       0.99      1.00      0.99       375
weighted avg       0.99      0.99      0.99       375

